# 1. Dimensionando y capturando

In [2]:
import cv2
import os
import random
import string
import matplotlib.pyplot as plt

Configurar video file: Vid_file

In [ ]:
vcap = cv2.VideoCapture('Vid_file')  # 0=camera

if vcap.isOpened(): 
    # get vcap property 
    width  = vcap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
    # or
    width  = vcap.get(3)  # float `width`
    height = vcap.get(4)  # float `height`

    # it gives me 0.0 :/
    fps = vcap.get(cv2.CAP_PROP_FPS)
    
    print(f"Width: {width}, Height: {height}, FPS: {fps}")
else:
    print("Error: No se pudo abrir el archivo de video.")
    

## Creando columna de control recortando el video

Configurar input_video_path y output_video_path
asi como también el ROI

In [ ]:

# Ruta al video de origen
input_video_path = 'input_video_path.mp4'
# Ruta al nuevo video que se va a crear
output_video_path = 'output_video_path.mp4'

# Coordenadas de la región de interés (ROI)
x, y, width, height = 200, 400, 600, 250  # (x, y) es la esquina superior izquierda y (width, height) es el tamaño de la ROI

# Cargar el video de origen
cap = cv2.VideoCapture(input_video_path)

# Obtener la información del video original
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Definir el codec y crear el objeto VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para formato mp4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Recortar la región de interés
    roi = frame[y:y+height, x:x+width]
    
    # Escribir el frame recortado en el nuevo video
    out.write(roi)

# Liberar los objetos VideoCapture y VideoWriter
cap.release()
out.release()

print("Proceso de recorte completado. Nuevo video guardado en:", output_video_path)


Proceso de recorte completado. Nuevo video guardado en: Mask_1.mp4


## Definiendo zona de interes y simulando señal del LiDAR

configurar ROI y color del rectangulo

In [ ]:
def draw_roi(frame, roi, color):
    # Get ROI coordinates
    x, y, w, h = roi
    # Draw rectangle on frame
    if color == 'blue':
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

Configurar input_video_path

In [ ]:
#Cambiando el tono
cap=cv2.VideoCapture('input_video_path.mp4')
current_char=-1
previous_char=-1
  
capturas=[]
while cap.isOpened():    
    ret,frame=cap.read()
    frame = cv2.resize(frame, None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
    c=cv2.waitKey(120)
    if c== ord('q'): #simulacion de captura manual con teclado letra 'c'
        break
    if c==ord('c'):
        capturas.append(frame)
        print(f"Cantidad de imagenes capturadas {len(capturas)}")

#Dibujando la region de interes
    roi = [75, 225, 200, 75]  
    draw_roi(frame, roi,'blue')

    cv2.imshow('Webcam',frame)
cap.release() 
# Closes all the frames 
cv2.destroyAllWindows()
print(f"Cantidad de imagenes capturadas {len(capturas)}")

## Mostrando las capturas y el acercamiento correspondiente

### Función mostrandoImagenesCapturadas

In [1]:
def onlyShowImag(img):
    plt.imshow(img)
    plt.axis('off')
    plt.show()

configurar rawDir

In [ ]:
def mostrandoImagenesCapturadas(capturas):
    rawDir="./rawDir"
    fig, axes = plt.subplots(1,len(capturas), figsize=(20,20))
    axes_flat=axes.flatten()
    count =0
    for i in range(len(capturas)):
        imagen_rgb = cv2.cvtColor(capturas[i],cv2.COLOR_BGR2RGB)
        axes_flat[i].imshow(imagen_rgb)
        axes_flat[i].set_title(f"Captura {count}")
        axes_flat[i].axis("off")
        count+=1
        output_path = os.path.join(rawDir, f'captura{i}.jpg')
        cv2.imwrite(output_path, capturas[i])
    onlyShowImag(imagen_rgb)

In [ ]:
def zoomImag(img,dir,zoom_factor):
    # Establecer una semilla aleatoria
    seed_value = random.random()
    random.seed(seed_value)

    # Generar números aleatorios

    rand_mark=random.randint(1, 1000)
    x, y, w, h = 75, 225, 200, 75
    roi = img[y:y+round(h), x:x+round(w)]
  
  #Nombrando cada imagen con nombres aleatorios y guardandolas en output_path
    zoomed_roi_save = cv2.resize(roi, (w * zoom_factor, h * zoom_factor), interpolation=cv2.INTER_LINEAR)
    zoomed_roi_show=  cv2.cvtColor(zoomed_roi_save,cv2.COLOR_BGR2RGB)
    output_path = os.path.join(dir, f'captura{i}_{rand_mark}.jpg')

    cv2.imwrite(output_path, zoomed_roi_save)
    onlyShowImag(zoomed_roi_show)
    return zoomed_roi_save

In [ ]:

def split_image(img, subimages_dir, count, grid_size=(5, 3)):

    # Crear la carpeta subimages si no existe
    if not os.path.exists(subimages_dir):
        os.makedirs(subimages_dir)

    # Obtener dimensiones de la imagen
    height, width, _ = img.shape
    rows, cols = grid_size
    
    # Calcular tamaño de las subimágenes
    sub_height = height // rows
    sub_width = width // cols

    # Dividir y guardar subimágenes
    for r in range(rows):
        for c in range(cols):
            # Definir la región de interés (ROI)
            y1 = r * sub_height
            y2 = (r + 1) * sub_height
            x1 = c * sub_width
            x2 = (c + 1) * sub_width
            roi = img[y1:y2, x1:x2]
            

            # Generar un nombre aleatorio para la subimagen
            rand_name = 'cap_'+ str(count)+"_"+''.join(random.choices(string.digits, k=8)) + ".jpg"
            output_path = os.path.join(subimages_dir, rand_name)

            # Guardar la subimagen
            cv2.imwrite(output_path, roi)

    print(f"Subimágenes guardadas en la carpeta '{subimages_dir}'")

Configurar subimages_dir y output_dir

In [ ]:
#Recordando que CV carga las imagenes en BGR y no RGB por eso hacemos la conversion

output_dir = 'output_dir'
os.makedirs(output_dir, exist_ok=True)
subimages_dir = './subimages_dir'

if len(capturas)>1:

    mostrandoImagenesCapturadas(capturas)

    for i in range(len(capturas)):

        zoomed=zoomImag(capturas[i],output_dir,zoom_factor = 30)

        split_image(zoomed, subimages_dir,i)
else:

    imagen_rgb = cv2.cvtColor(capturas[0],cv2.COLOR_BGR2RGB)

    zoomed=zoomImag(capturas[0],output_dir,zoom_factor = 30)

    count=0

    split_image(zoomed, subimages_dir,count)

onlyShowImag(imagen_rgb)